In [24]:
!pip install pyspark

In [25]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [26]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
home_sales_df.show()



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [27]:
# 2. Create a temporary view of the DataFrame.

home_sales_df.createOrReplaceTempView('home_sales')

In [28]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""select Round(Avg(price),2) as average_price ,date_built  from home_sales where bedrooms=4 group by date_built""").show()


+-------------+----------+
|average_price|date_built|
+-------------+----------+
|    296050.24|      2016|
|    298233.42|      2012|
|    296576.69|      2017|
|    299073.89|      2014|
|    299999.39|      2013|
|     302141.9|      2011|
|    307908.86|      2015|
|    296800.75|      2010|
+-------------+----------+



In [29]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""select Round(Avg(price),2) as average_price ,date_built  from home_sales where bedrooms=3 and bathrooms=3 group by date_built""").show()


+-------------+----------+
|average_price|date_built|
+-------------+----------+
|    290555.07|      2016|
|    293683.19|      2012|
|    292676.79|      2017|
|    290852.27|      2014|
|    295962.27|      2013|
|    291117.47|      2011|
|     288770.3|      2015|
|    292859.62|      2010|
+-------------+----------+



In [30]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""select Round(Avg(price),2) as average_price ,
date_built
from home_sales
where bedrooms=3 and bathrooms=3 and floors=2 and sqft_lot>=2000
group by date_built""").show()


+-------------+----------+
|average_price|date_built|
+-------------+----------+
|     287812.6|      2016|
|    295858.68|      2012|
|    282026.59|      2017|
|    294195.13|      2014|
|    295142.13|      2013|
|    281413.45|      2011|
|    291788.36|      2015|
|    280447.23|      2010|
+-------------+----------+



In [31]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""select view,Round(Avg(price),2) as average_price ,
date_built
from home_sales
where price >= 350000
group by date_built,view""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+----------+
|view|average_price|date_built|
+----+-------------+----------+
|   2|    402938.65|      2014|
|  91|    1229833.2|      2012|
|  29|    396646.37|      2015|
|  64|     821564.5|      2010|
|  49|     405113.2|      2016|
|  99|    1286284.5|      2013|
|  11|    402260.46|      2014|
|  11|    403923.05|      2012|
|  89|   1188010.22|      2012|
|  15|    406913.53|      2010|
|  45|    396184.61|      2012|
|  49|    404430.18|      2014|
|  50|    436670.66|      2010|
|  44|    412683.27|      2010|
|  26|    407449.25|      2014|
|  71|    785892.67|      2010|
|  66|     556223.0|      2014|
|  46|    391369.25|      2011|
|  36|    403731.21|      2013|
|   8|    403550.45|      2014|
+----+-------------+----------+
only showing top 20 rows

--- 1.1644182205200195 seconds ---


In [32]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [33]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [34]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""select view,Round(Avg(price),2) as average_price ,
date_built
from home_sales
where price >= 350000
group by date_built,view""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+----------+
|view|average_price|date_built|
+----+-------------+----------+
|   2|    402938.65|      2014|
|  91|    1229833.2|      2012|
|  29|    396646.37|      2015|
|  64|     821564.5|      2010|
|  49|     405113.2|      2016|
|  99|    1286284.5|      2013|
|  11|    402260.46|      2014|
|  11|    403923.05|      2012|
|  89|   1188010.22|      2012|
|  15|    406913.53|      2010|
|  45|    396184.61|      2012|
|  49|    404430.18|      2014|
|  50|    436670.66|      2010|
|  44|    412683.27|      2010|
|  26|    407449.25|      2014|
|  71|    785892.67|      2010|
|  66|     556223.0|      2014|
|  46|    391369.25|      2011|
|  36|    403731.21|      2013|
|   8|    403550.45|      2014|
+----+-------------+----------+
only showing top 20 rows

--- 0.6440944671630859 seconds ---


In [35]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("P_home_sales")

In [36]:
# 11. Read the formatted parquet data.
p_home_sales_df=spark.read.parquet('P_home_sales')

In [37]:
# 12. Create a temporary table for the parquet data.
p_home_sales_df.createOrReplaceTempView('p_temp_home_sales')

In [38]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""select view,Round(Avg(price),2) as average_price ,
date_built
from p_temp_home_sales
where price >= 350000
group by date_built,view""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+----------+
|view|average_price|date_built|
+----+-------------+----------+
|  45|    396184.61|      2012|
|  69|    564801.25|      2017|
|  20|    402351.32|      2014|
|  74|    716809.25|      2015|
|  65|     992484.0|      2014|
|  94|   1031061.71|      2012|
|  79|    1094288.5|      2015|
|   0|    401229.13|      2012|
|  74|     750350.0|      2017|
|  93|    659662.25|      2012|
|  19|    400568.89|      2014|
|  65|     639283.5|      2012|
|  91|    1229833.2|      2012|
|  98|    1176968.5|      2014|
|   3|    401942.17|      2015|
|  41|    403644.22|      2014|
|  20|    395093.88|      2017|
|  54|     766411.0|      2012|
|  46|    401401.35|      2017|
|  38|    399901.24|      2015|
+----+-------------+----------+
only showing top 20 rows

--- 0.8278937339782715 seconds ---


In [39]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [40]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False